Intstalling Huggingface Transformers, Datasets, and Evaluate

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate
!pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Research/NLP/Project/

Mounted at /content/drive


#### Tokenize function to tokenize the input data

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

def nltk_tokenization(text):
  """
  text: raw text
  return a string after tokenization
  """
  return " ".join(word_tokenize(text))


## tOK

In [ ]:
import string
import re
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
/tmp/ipython-input-1190099680.py:3: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', text)


In [ ]:
t = clean_text("he's gone's as hekk''''''' ddfd fd")
print(t)

['he', 'gone', 'hekk', 'ddfd', 'fd']


## rEAD

In [ ]:
import csv
def read_data(in_file, delim=','):
  print(f"Reading file {in_file}.......")
  output_data = []
  with open(in_file, 'r') as f:
    reader = csv.reader(f, delimiter=delim)
    header = next(reader)
    output_data.append(['sentence', 'label'])
    for row in reader:
      if len(row) >=2:
        tokenized_text = nltk_tokenization(row[1])
        label = {"None": 0, "Religious Hate": 1, "Sexism": 2, "Political Hate": 3, "Profane": 4, "Abusive": 5}[row[2]]
        output_data.append([tokenized_text, label])
  return output_data

In [ ]:
# import csv
# def read_data_t(in_file, delim=','):
#   print(f"Reading file {in_file}.......")
#   output_data = []
#   with open(in_file, 'r') as f:
#     reader = csv.reader(f, delimiter=delim)
#     header = next(reader)
#     output_data.append(['sentence', 'label'])
#     for row in reader:
#       if len(row) >=2:
#         tokenized_text = nltk_tokenization(row[1])
#         output_data.append([tokenized_text, 1])
#   return output_data


#### Save the toeknized data

In [ ]:
def write_file(data_list:list, output_file, delim=','):
  print(f"Writing file {output_file}.......")
  with open(output_file, 'w') as f:
    writer = csv.writer(f, delimiter=delim)
    for row in data_list:
      writer.writerow(row)

In [ ]:
!mkdir data/sub-task-1a/tokenized

mkdir: cannot create directory ‘data/sub-task-1a/tokenized’: File exists


In [ ]:
tokenized_texts = read_data('./data/sub-task-1a/train.tsv', delim='\t')
write_file(tokenized_texts, './data/sub-task-1a/tokenized/train.csv', delim=',')

tokenized_texts = read_data('./data/sub-task-1a/dev.tsv', delim='\t')
write_file(tokenized_texts, './data/sub-task-1a/tokenized/dev.csv', delim=',')

tokenized_texts = read_data('./data/sub-task-1a/test.tsv', delim='\t')
write_file(tokenized_texts, './data/sub-task-1a/tokenized/test.csv', delim=',')

Reading file ./data/sub-task-1a/train.tsv.......
Writing file ./data/sub-task-1a/tokenized/train.csv.......
Reading file ./data/sub-task-1a/dev.tsv.......
Writing file ./data/sub-task-1a/tokenized/dev.csv.......
Reading file ./data/sub-task-1a/test.tsv.......
Writing file ./data/sub-task-1a/tokenized/test.csv.......


In [ ]:
!mkdir output-subtask-1a

mkdir: cannot create directory ‘output-subtask-1a’: File exists


##rUN the script

In [ ]:
!python scripts/run_glue_v2.py \
  --model_name_or_path distilbert-base-uncased \
  --train_file ./data/sub-task-1a/tokenized/train.csv \
  --validation_file ./data/sub-task-1a/tokenized/dev.csv \
  --test_file ./data/sub-task-1a/tokenized/test.csv \
  --do_train \
  --do_eval \
  --do_predict \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --output_dir ./output-subtask-1a/output_distilbert-base-uncased/ \
  --overwrite_output_dir

2025-11-27 21:38:30.606985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764279510.633808   17714 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764279510.642147   17714 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764279510.671250   17714 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764279510.671280   17714 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764279510.671286   17714 computation_placer.cc:177] computation placer alr

## Calculating Classification Report

In [ ]:
from sklearn import metrics

def calculate_performance(y_true, y_pred, labels):
    """
    Calculating performances of our model
    :param y_true: actual labels in test set
    :param y_pred: predicted labels
    :param labels:
    :return: accuracy, precision, recall, f1 score and classification report
    """
    (acc, P, R, F1) = (0.0, 0.0, 0.0, 0.0)
    acc = metrics.accuracy_score(y_true, y_pred)
    P = metrics.precision_score(y_true, y_pred, average='weighted')
    R = metrics.recall_score(y_true, y_pred, average='weighted')
    F1 = metrics.f1_score(y_true, y_pred, average='weighted')
    report = metrics.classification_report(y_true, y_pred, target_names=labels)

    return acc * 100, P * 100, R * 100, F1 * 100, report

#### Reading files for calculate the performance on test data

In [ ]:
import csv
def read_labels(filename):
    labels = []

    # Define the mapping once
    label_map = {
        "0": "None",
        "1": "Religious Hate",
        "2": "Sexism",
        "3": "Political Hate",
        "4": "Profane",
        "5": "Abusive"
    }
    # File Handling
    if filename.endswith('.csv'):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader, None)
            for row in reader:
                if len(row) < 2:
                    continue
                # Assuming the prediction is in column 2 (index 1)
                pred_key = row[1].strip()
                if pred_key in label_map:
                    labels.append(label_map[pred_key])
    else:
        # Using encoding='utf-8' for safety
        with open(filename, 'r', encoding='utf-8') as f:
            # Read all lines and skip the header
            lines = f.readlines()[1:]
            for line in lines:
                line = line.strip()
                if not line: # Skip potential empty lines (e.g., trailing newline)
                    continue
                parts = line.split("\t")
                if len(parts) < 2:
                    continue
                lab = parts[1] # e.g., '0'
                if lab in label_map:
                    labels.append(label_map[lab])
    return labels

#### Performances on Test set

In [ ]:
y_test_true = read_labels('./data/sub-task-1a/tokenized/test.csv')                                #gold labels
y_test_pred = read_labels('./output-subtask-1a/output_distilbert-base-cased/predict_results.txt') #change the output file for different models
assert len(y_test_true) == len(y_test_pred)
target_labels = ["None", "Religious Hate", "Sexism", "Political Hate", "Profane", "Abusive"]
acc, precision, recall, F1, report = calculate_performance(y_test_true, y_test_pred, target_labels)
result = str("{0:.4f}".format(acc)) + "\t" + str("{0:.4f}".format(precision)) + "\t" + str(
    "{0:.4f}".format(recall)) + "\t" + str("{0:.4f}".format(F1)) + "\n"

print("Test set:\t Acc\tPrecision\tRecall\tF1\n" + result)
print(report)

Test set:	 Acc	Precision	Recall	F1
62.0294	56.2471	62.0294	56.7535

                precision    recall  f1-score   support

          None       0.37      0.18      0.24       753
Religious Hate       0.66      0.91      0.77      1939
        Sexism       0.52      0.31      0.39       410
Political Hate       0.60      0.36      0.45       221
       Profane       0.00      0.00      0.00        68
       Abusive       0.00      0.00      0.00         9

      accuracy                           0.62      3400
     macro avg       0.36      0.29      0.31      3400
  weighted avg       0.56      0.62      0.57      3400



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m